Here I want to measure fairness using the generalization of disparate impact found [here](https://drive.google.com/open?id=1c8jUcH3wCoFMf6YAti4QWYFqwhlKYhfI).

In [41]:
import numpy as np
import pandas as pd
import math

In [ ]:
def logDI(df, labels_col, terms):
    
    logDI_arr = np.empty((len(terms)), len(terms))
    
    for i in range(len(terms)):
        for j in range(len(terms)):
            
            logDI[i, j] =  math.log(len(df[(df['i']==True) & (df[labels_col]==True)]) / len(df[df['i']==True])) \
                            - math.log(len(df[(df['j']==True) & (df[labels_col]==True)]) / len(df[df['j']==True]))
                
    logDI = logDI_arr.sum()
    return logDI